$ \huge{1.}$ Let C[_w_,_m_] be the cost of transporting one truck-load of EVs from warehouse _w_ to market _m_, where _w_ and _m_ both go from 1 to 8. The cost matrix is already given. Let $x_{wm}$ represent the number of truck-loads of EVs from warehouse _w_ to market _m_. \\
 \\
Therefore, our objective function becomes \\
 \\
**min $\sum_{w=1}^{8} \sum_{m=1}^{8} C[w,m]*x_{ij}$** \\
 \\
subject to the following constraints \\
 \\
**Demand constraint:** **$\sum_{i=1}^{8} x_{ij}$ = $d_{j}$ for all j** ;where $d_{j}$ is the demand at the j _th_ warehouse \\
 \\
**Stock Availability constraint:** **$\sum_{j=1}^{8} x_{ij}$ <= $a_{i}$ for all i** ; where $a_{i}$ is the stock availability at the i _th_ warehouse. \\
 \\
Further, $x_{ij}$ >= 0 for all i and j as these variables denote number of truck-loads, so they must be non-negative. Further, these variables are of integer type.

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 6.1 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 


In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
data_csvfile = pd.read_csv('lab6_ex1.csv') #data_csvfile is a pd dataframe

In [6]:
data_csvfile.index #indexing the rows

RangeIndex(start=0, stop=9, step=1)

In [11]:
# print('num rows:', len(data_csvfile.index)) #counting the number of rows, length is 4 and not 5 as first row is considered as a 'header row'/descriptive row

In [9]:
#to get the row contents corresponding to Ahmedabad
# print(data_csvfile.iloc[0,:])

In [10]:
# data_csvfile.columns

In [12]:
# print('num  columns: ',len(data_csvfile.columns))

In [13]:
#to get the column contents corresponding to Hassan
# print(data_csvfile.iloc[:,1])#Numbers on left correspond to row numbers

In [14]:
# create a model
model_lab5_ex1 = ConcreteModel()

In [15]:
# we can set certain values using python variables
M = len(data_csvfile.index)-1 #we declare a Python variable M denoting the number of steel plants  
print('num warehouses: ',M)

num warehouses:  8


In [16]:
N = len(data_csvfile.columns)-2 #we declare a Python variable N denoting the number of hubs
print('num markets: ',N)

num markets:  8


In [17]:
row_indices = range(M)
col_indices = range(N)

print(row_indices)
print(col_indices)

range(0, 8)
range(0, 8)


In [18]:
#to get the contents of costs alone from the file we use the following
for row_idx in row_indices:
  for col_idx in col_indices:
    print('cost[',row_idx,'][',col_idx+1,']:',data_csvfile.iloc[row_idx, col_idx+1])

cost[ 0 ][ 1 ]: 827
cost[ 0 ][ 2 ]: 817
cost[ 0 ][ 3 ]: 1470
cost[ 0 ][ 4 ]: 2272
cost[ 0 ][ 5 ]: 1615
cost[ 0 ][ 6 ]: 3143
cost[ 0 ][ 7 ]: 1264
cost[ 0 ][ 8 ]: 4465
cost[ 1 ][ 1 ]: 559
cost[ 1 ][ 2 ]: 1832
cost[ 1 ][ 3 ]: 751
cost[ 1 ][ 4 ]: 3772
cost[ 1 ][ 5 ]: 637
cost[ 1 ][ 6 ]: 3773
cost[ 1 ][ 7 ]: 1856
cost[ 1 ][ 8 ]: 5459
cost[ 2 ][ 1 ]: 1109
cost[ 2 ][ 2 ]: 2443
cost[ 2 ][ 3 ]: 577
cost[ 2 ][ 4 ]: 3865
cost[ 2 ][ 5 ]: 798
cost[ 2 ][ 6 ]: 3521
cost[ 2 ][ 7 ]: 1984
cost[ 2 ][ 8 ]: 5303
cost[ 3 ][ 1 ]: 2943
cost[ 3 ][ 2 ]: 1471
cost[ 3 ][ 3 ]: 4676
cost[ 3 ][ 4 ]: 1337
cost[ 3 ][ 5 ]: 2702
cost[ 3 ][ 6 ]: 2741
cost[ 3 ][ 7 ]: 2482
cost[ 3 ][ 8 ]: 4148
cost[ 4 ][ 1 ]: 3742
cost[ 4 ][ 2 ]: 3089
cost[ 4 ][ 3 ]: 2931
cost[ 4 ][ 4 ]: 3199
cost[ 4 ][ 5 ]: 2226
cost[ 4 ][ 6 ]: 1739
cost[ 4 ][ 7 ]: 1691
cost[ 4 ][ 8 ]: 2128
cost[ 5 ][ 1 ]: 2353
cost[ 5 ][ 2 ]: 1160
cost[ 5 ][ 3 ]: 3295
cost[ 5 ][ 4 ]: 368
cost[ 5 ][ 5 ]: 2369
cost[ 5 ][ 6 ]: 2365
cost[ 5 ][ 7 ]: 1293
cost[ 5 ][ 8 ]: 3642


In [19]:
#declare the decision variables in the model
model_lab5_ex1.x = Var(row_indices, col_indices, domain= NonNegativeReals)

In [20]:
model_lab5_ex1.objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model_lab5_ex1.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [21]:
model_lab5_ex1.constraints = ConstraintList()

In [23]:
#to get the supply (or production) limit at warehouses from the file we can use the following
# print(data_csvfile.iloc[:8,N+1])

In [24]:
#to get the demands at markets from the file we can use the following
# print(data_csvfile.iloc[M,1:N+1])

In [25]:
#first We will add the supply (or production) limit constraints
for i in row_indices:
  model_lab5_ex1.constraints.add(expr=sum(model_lab5_ex1.x[i,j] for j in col_indices) <= data_csvfile.iloc[i,N+1])

In [26]:
#Now we will add the demand constraints
for j in col_indices:
  model_lab5_ex1.constraints.add(expr=sum(model_lab5_ex1.x[i,j] for i in row_indices) == data_csvfile.iloc[M,j+1])

In [27]:
model_lab5_ex1.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
    

In [28]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [29]:
opt_cbc = SolverFactory('cbc')

In [30]:
result = opt_cbc.solve(model_lab5_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [31]:
# display solution
print('\nObjective = ', model_lab5_ex1.objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    print('Num of EVs between ',data_csvfile['WH_Markets'][i],' and ', data_csvfile.columns[j+1],':', model_lab5_ex1.x[i,j].value)

print('\nConstraints')
model_lab5_ex1.constraints.display()


Objective =  2192870.0

Decision Variables
Num of EVs between  Ahmedabad  and  Hassan : 0.0
Num of EVs between  Ahmedabad  and  Jodhpur : 300.0
Num of EVs between  Ahmedabad  and  Trichy : 0.0
Num of EVs between  Ahmedabad  and  Simla : 0.0
Num of EVs between  Ahmedabad  and  Vijayawada : 0.0
Num of EVs between  Ahmedabad  and  Dibrugarh : 45.0
Num of EVs between  Ahmedabad  and  Raipur : 5.0
Num of EVs between  Ahmedabad  and  Agartala : 0.0
Num of EVs between  Bengaluru  and  Hassan : 0.0
Num of EVs between  Bengaluru  and  Jodhpur : 0.0
Num of EVs between  Bengaluru  and  Trichy : 120.0
Num of EVs between  Bengaluru  and  Simla : 0.0
Num of EVs between  Bengaluru  and  Vijayawada : 65.0
Num of EVs between  Bengaluru  and  Dibrugarh : 0.0
Num of EVs between  Bengaluru  and  Raipur : 0.0
Num of EVs between  Bengaluru  and  Agartala : 0.0
Num of EVs between  Chennai  and  Hassan : 0.0
Num of EVs between  Chennai  and  Jodhpur : 0.0
Num of EVs between  Chennai  and  Trichy : 305.0
Num 

$ \huge{7.}$ The number of truck-loads of EVs that are transported from warehouses to markets are: \\
From **Ahmedabad** to **Jodhpur**: 300 \\
From **Ahmedabad** to **Dibrugarh**: 45 \\
From **Ahmedabad** to **Raipur**: 5 \\
From **Bengaluru** to **Trichy**: 120 \\
From **Bengaluru** to **Vijayawada**: 65 \\
From **Chennai** to **Trichy**: 305 \\
From **Delhi** to **Simla**: 95 \\
From **Delhi** to **Dibrugarh**: 85 \\
From **Kolkata** to **Dibrugarh**: 95 \\
From **Kolkata** to **Agartala**: 110 \\
From **Lucknow** to **Simla**: 175 \\
From **Mumbai** to **Hassan**: 205 \\
From **Mumbai** to **Raipur**: 120 \\
From **Hyderabad** to **Vijayawada**: 310 \\
From **Hyderabad** to **Raipur**: 60 \\
 \\
The **cost** incurred is **2192870.0 units.**

$ \huge{8.}$ The Lucknow-Simla link is disrupted and no transportation is possible on the Lucknow-Simla links. To incorporate this modification, we change the value associated with the Lucknow-Simla link. We assign a very high cost to this link so it may not be considered in the optimal solution, which aims at minimizing the cost.

In [32]:
data_csvfile.iloc[5,4]=9999999 #Changing the cost associated with the Lucknow-Simla link

In [33]:
model_lab5_ex1.objective.deactivate()

In [34]:
model_lab5_ex1.new_objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model_lab5_ex1.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [35]:
result = opt_cbc.solve(model_lab5_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [36]:
# display solution
print('\nObjective = ', model_lab5_ex1.new_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    print('Num of EVs between ',data_csvfile['WH_Markets'][i],' and ', data_csvfile.columns[j+1],':', model_lab5_ex1.x[i,j].value)

print('\nConstraints')
model_lab5_ex1.constraints.display()


Objective =  2380570.0

Decision Variables
Num of EVs between  Ahmedabad  and  Hassan : 0.0
Num of EVs between  Ahmedabad  and  Jodhpur : 300.0
Num of EVs between  Ahmedabad  and  Trichy : 0.0
Num of EVs between  Ahmedabad  and  Simla : 0.0
Num of EVs between  Ahmedabad  and  Vijayawada : 0.0
Num of EVs between  Ahmedabad  and  Dibrugarh : 0.0
Num of EVs between  Ahmedabad  and  Raipur : 50.0
Num of EVs between  Ahmedabad  and  Agartala : 0.0
Num of EVs between  Bengaluru  and  Hassan : 0.0
Num of EVs between  Bengaluru  and  Jodhpur : 0.0
Num of EVs between  Bengaluru  and  Trichy : 120.0
Num of EVs between  Bengaluru  and  Simla : 0.0
Num of EVs between  Bengaluru  and  Vijayawada : 65.0
Num of EVs between  Bengaluru  and  Dibrugarh : 0.0
Num of EVs between  Bengaluru  and  Raipur : 0.0
Num of EVs between  Bengaluru  and  Agartala : 0.0
Num of EVs between  Chennai  and  Hassan : 0.0
Num of EVs between  Chennai  and  Jodhpur : 0.0
Num of EVs between  Chennai  and  Trichy : 305.0
Num 

$ \huge{9.}$ The number of truck-loads of EVs that are transported from warehouses to markets are: \\
From **Ahmedabad** to **Jodhpur**: 300 \\
From **Ahmedabad** to **Raipur**: 50 \\
From **Bengaluru** to **Trichy**: 120 \\
From **Bengaluru** to **Vijayawada**: 65 \\
From **Chennai** to **Trichy**: 305 \\
From **Delhi** to **Simla**: 180 \\
From **Kolkata** to **Dibrugarh**: 95 \\
From **Kolkata** to **Agartala**: 110 \\
From **Lucknow** to **Dibrugarh**: 130 \\
From **Lucknow** to **Raipur**: 45 \\
From **Mumbai** to **Hassan**: 205 \\
From **Mumbai** to **Simla**: 90 \\
From **Mumbai** to **Raipur**: 30 \\
From **Hyderabad** to **Vijayawada**: 310 \\
From **Hyderabad** to **Raipur**: 60 \\
 \\
The **total cost** incurred is **2380570.0 units.** \\
 \\
**Observations:** The original cost associated with the Lucknow-Simla route was part of our optimal solution. As this cost was changed and increased to a very high value, the new total cost obtained is higher than the cost obtained previously as one of the links that was a part of the optimal solution was changed. To make up for this while trying to keep the cost minimum, other cities had to send some of their supplies to Simla instead of sending them to the cities with which the lowest costs were associated. This is another reason why the total cost incurred went up.

$ \huge{10.}$ Now, along with the Lucknow-Simla route, there are disruptions in Kolkata-Dibrugarh and Delhi-Raipur links as well. Thus, we repeat the same task of assigning very high costs to these two links so that our optimal solution does not consider these routes as viable options.

In [37]:
data_csvfile.iloc[4,6]=9999999 #Assinging high cost to Kolkata-Dibrugarh route
data_csvfile.iloc[3,7]=9999999 #Assinging high cost to Delhi-Raipur route

In [38]:
model_lab5_ex1.new_objective.deactivate()

In [39]:
model_lab5_ex1.new2_objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model_lab5_ex1.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [40]:
result = opt_cbc.solve(model_lab5_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [41]:
# display solution
print('\nObjective = ', model_lab5_ex1.new2_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    print('Num of EVs between ',data_csvfile['WH_Markets'][i],' and ', data_csvfile.columns[j+1],':', model_lab5_ex1.x[i,j].value)

print('\nConstraints')
model_lab5_ex1.constraints.display()


Objective =  2518200.0

Decision Variables
Num of EVs between  Ahmedabad  and  Hassan : 0.0
Num of EVs between  Ahmedabad  and  Jodhpur : 300.0
Num of EVs between  Ahmedabad  and  Trichy : 0.0
Num of EVs between  Ahmedabad  and  Simla : 0.0
Num of EVs between  Ahmedabad  and  Vijayawada : 0.0
Num of EVs between  Ahmedabad  and  Dibrugarh : 50.0
Num of EVs between  Ahmedabad  and  Raipur : 0.0
Num of EVs between  Ahmedabad  and  Agartala : 0.0
Num of EVs between  Bengaluru  and  Hassan : 0.0
Num of EVs between  Bengaluru  and  Jodhpur : 0.0
Num of EVs between  Bengaluru  and  Trichy : 120.0
Num of EVs between  Bengaluru  and  Simla : 0.0
Num of EVs between  Bengaluru  and  Vijayawada : 65.0
Num of EVs between  Bengaluru  and  Dibrugarh : 0.0
Num of EVs between  Bengaluru  and  Raipur : 0.0
Num of EVs between  Bengaluru  and  Agartala : 0.0
Num of EVs between  Chennai  and  Hassan : 0.0
Num of EVs between  Chennai  and  Jodhpur : 0.0
Num of EVs between  Chennai  and  Trichy : 305.0
Num 

$ \huge{11.}$ The number of truck-loads of EVs that are transported from warehouses to markets are: \\
From **Ahmedabad** to **Jodhpur**: 300 \\
From **Ahmedabad** to **Dibrugarh**: 50 \\
From **Bengaluru** to **Trichy**: 120 \\
From **Bengaluru** to **Vijayawada**: 65 \\
From **Chennai** to **Trichy**: 305 \\
From **Delhi** to **Simla**: 180 \\
From **Kolkata** to **Raipur**: 95 \\
From **Kolkata** to **Agartala**: 110 \\
From **Lucknow** to **Dibrugarh**: 175 \\
From **Mumbai** to **Hassan**: 205 \\
From **Mumbai** to **Simla**: 90 \\
From **Mumbai** to **Raipur**: 30 \\
From **Hyderabad** to **Vijayawada**: 310 \\
From **Hyderabad** to **Raipur**: 60 \\
 \\
The **total cost** incurred is **2518200.0 units.** \\
 \\
**Observations:** As stated previously, as some of the costs were increased to a very high value, thus deeming them unfit to be a part of the optimal solution, the new total cost obtained is higher than the previous total cost. Given the modifications made, the cost could be minimized only if other cities send some of the supplies to the markets associated with routes whose values were changed, thus leading to an increase in the price that the warehouse cities incur on an individual level, resulting in the increment of the total cost incurred.